In [3]:
import random

# Función objetivo a minimizar
def f(x, y):
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

# Parámetros del algoritmo genético
pop_size = 100  # Tamaño de la población
num_generations = 100  # Número de generaciones
mutation_rate = 0.1  # Tasa de mutación
elitism_count = 10  # Número de mejores individuos a mantener en cada generación

# Rango de valores permitidos para x e y
x_min, x_max = -4.5, 4.5  
y_min, y_max = -4.5, 4.5   

# Inicializar la población cerca del valor óptimo
population = [(random.uniform(x_min, x_max), random.uniform(y_min, y_max)) for _ in range(pop_size)]

# Función para evaluar la aptitud de un individuo
def evaluate(individual):
    x, y = individual
    return f(x, y)

# Ejecutar el algoritmo genético
for generation in range(num_generations):
    # Evaluar la aptitud de la población actual
    fitness_scores = [evaluate(individual) for individual in population]

    # Seleccionar a los mejores individuos (elitismo)
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population))]
    elite = sorted_population[:elitism_count]

    # Generar nueva población mediante cruzamiento y mutación
    new_population = elite[:]
    while len(new_population) < pop_size:
        # Seleccionar dos padres de manera aleatoria entre la élite
        parent1, parent2 = random.sample(elite, 2)

        # Realizar cruzamiento aleatorio
        child = [parent1[i] if random.random() < 0.5 else parent2[i] for i in range(2)]

        # Aplicar mutación
        if random.random() < mutation_rate:
            mutation_index = random.randint(0, 1)
            mutation_value = random.uniform(-0.5, 0.5)
            child[mutation_index] += mutation_value

            # Asegurarse de que el hijo esté dentro del rango permitido
            child[mutation_index] = max(min(child[mutation_index], x_max), x_min)

        new_population.append(tuple(child))

    # Actualizar la población
    population = new_population

    # Obtener la mejor solución encontrada en esta época
    best_individual = elite[0]
    best_fitness = evaluate(best_individual)

    # Imprimir la mejor solución de esta época
    print(f"Época {generation + 1}:")
    print(f"x = {best_individual[0]}")
    print(f"y = {best_individual[1]}")
    print(f"Valor de la función: {best_fitness}")
    print()
    

# Obtener la mejor solución encontrada al final del algoritmo
best_individual = elite[0]
best_fitness = evaluate(best_individual)

print("Mejor solución encontrada al final:")
print(f"x = {best_individual[0]}")
print(f"y = {best_individual[1]}")
print(f"Valor de la función: {best_fitness}")


Época 1:
x = 2.125106672065902
y = 0.1585580639251729
Valor de la función: 0.37326515423738554

Época 2:
x = 2.125106672065902
y = 0.1585580639251729
Valor de la función: 0.37326515423738554

Época 3:
x = 2.5107568118118513
y = 0.2895706079092786
Valor de la función: 0.11371449431029637

Época 4:
x = 2.5107568118118513
y = 0.37618999532469843
Valor de la función: 0.07478901312981208

Época 5:
x = 2.5107568118118513
y = 0.37618999532469843
Valor de la función: 0.07478901312981208

Época 6:
x = 2.5228692960196732
y = 0.37618999532469843
Valor de la función: 0.06843417263804476

Época 7:
x = 2.5654638286846065
y = 0.37618999532469843
Valor de la función: 0.050799700121908375

Época 8:
x = 2.678243889456437
y = 0.37618999532469843
Valor de la función: 0.03954822047689977

Época 9:
x = 2.6565631754116135
y = 0.37618999532469843
Valor de la función: 0.0377165806766913

Época 10:
x = 2.6565631754116135
y = 0.37618999532469843
Valor de la función: 0.0377165806766913

Época 11:
x = 2.6565631754